### 3D Audio Assignment 4
#### Crosstalk Cancellation 
#### Kol Zuo
#### cz1565@nyu.edu

In [1]:
'''
    import packages and modules
''' 

'\n    import packages and modules\n'

In [2]:
import numpy as np
import scipy as sci
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib
import matplotlib.pyplot as plt
import math

### Implement the crosstalk cancellation using the straight forward principle of recursive cancellation between ipsilateral and contralateral ears

In [3]:
# Initalize several parameters
spkr_to_spkr = 0.5
ear_to_ear = 0.224
listnr_to_spkr = 0.8
# compute several corresponding physical parameters
# suppose the plane of the center of the head lies on the mid point between two speakers
s = spkr_to_spkr / 2
r = ear_to_ear / 2
d = s - r
h = np.sqrt(listnr_to_spkr**2 - s**2)
l = np.sqrt(s**2 + h**2)
theta = math.asin(h/l)
L_ipsi = np.sqrt(d**2 + h**2)
L_cont = L_ipsi+ r * (np.pi - 2 * theta)

In [4]:
# function for integral delay
# here just use circular delay for convenience
# credits to "pi19404" on Github for the idea of fractional delay
def delay(signal, N):
    if N == 0:
        return signal;    
    if N >= len(signal):
        N = N%len(signal)
    if N <0:
        N =N %len(signal)
    d = signal[len(signal)-N:len(signal)];#numpy.zeros((1,N+1));    
    # wrap around
    output = np.append(d,signal[0:len(signal)-N])
    return output

In [5]:
# function for linear interpolation of fractional delay with N samples
def fraction_delay(signal, N):
    f, i = math.modf(N)
    signal = delay(signal, int(i)) # integral delay first
    output = sci.signal.convolve(signal, [f, 1-f]) # the linear interpolation for fractional delay
    return output

In [7]:
# function for computing head shadow coefficients
# credits to Digial Audio Effect by Udo Zolzer
def compute_headshadow_coefficients(theta, r, sr):
    theta = theta + math.pi / 2
    theta0 = 150 * math.pi / 180
    alpha_min = 0.05
    c = 343.2 # speed of sound.
    w0 = c / r
    alpha = (1 + alpha_min/2) + (1 - alpha_min/2) * math.cos(theta * math.pi / theta0)
    b = [(alpha+w0/sr)/(1+w0/sr), (-alpha+w0/sr)/(1+w0/sr)]
    a = [1, -(1-w0/sr)/(1+w0/sr)]
    return b, a

In [8]:
# function for recursive cancel
def recursive_cancel(signal, time_delay, ref, attenuation, headshadow, sr, threshold_db = -60):
    # delay and invert the signal (use fractional delay)
    delay_sample = int(np.ceil(sr * time_delay))
    cancel_signal = fraction_delay(signal, delay_sample)
    cancel_signal = cancel_signal * -1
    # then apply the headshadow filter and attenuate it
    cancel_signal = sci.signal.filtfilt(*headshadow, cancel_signal) * attenuation
    # get the max magnitude of cancel signal
    max_db = np.max(np.abs(cancel_signal))
    # compute the cancel db relative to reference max magnitude
    cancel_db = 20 * np.log10(max_db / ref)
    if cancel_db < threshold_db:
        return cancel_signal
    else:
        yield cancel_signal
        yield from recursive_cancel(cancel_signal, time_delay, ref, attenuation, headshadow, sr)

In [9]:
# function for cancel crosstalk
def cancel_crosstalk(signal, d1, d2, headshadow, sr):
    # assume the d1 would be ipsilateral signal and d2 would be contralateral signal
    c = 343.2
    delay_d = np.absolute(d2-d1)
    time_delay = delay_d / c
    # assume the attenuation would be
    attenuation = d1 / d2
    # get the reference max magnitude
    ref = np.max(np.abs(signal))
    cancel_signal = recursive_cancel(signal, time_delay, ref, attenuation, headshadow, sr)
    # "listalize" the generator
    cancel_sig = list(cancel_signal)
    # obtain the contralateral and ipsilateral cancel signal
    contralaterl = signal_sum(cancel_sig[0::2])
    ipisilateral = signal_sum(cancel_sig[1::2])
    
    return ipisilateral, contralaterl

In [10]:
# function for summing the signals
def signal_sum(signals):
    output = np.array([])
    max_len = 0
    for sig in signals:
        if len(sig) > max_len:
            len_diff = len(sig) - max_len 
            max_len = len(sig)
            output = np.pad(output, (0, len_diff), 'constant', constant_values = (0.0, 0.0))
        else:
            len_diff = max_len - len(sig)
            sig = np.pad(sig, (0, len_diff), 'constant', constant_values = (0.0, 0.0))
        output += sig
    return output

In [11]:
'''
    test the whole cross-talk cancellation process
'''

'\n    test the whole cross-talk cancellation process\n'

In [14]:
# test functions
file = 'test.wav'
y, sr = librosa.core.load(file, sr=44100, mono=False)
left = y[0]
right = y[1]

# compute the headshadow
headshadow = compute_headshadow_coefficients(theta, r, sr)
# compute the cancel crosstalk
L_left, L_right = cancel_crosstalk(left, L_ipsi, L_cont, headshadow, sr)
R_right, R_left = cancel_crosstalk(right, L_ipsi, L_cont, headshadow, sr)

left = signal_sum([L_left, R_left, left])
right = signal_sum([L_right, R_right, right])

# normalize
left = left / np.max(np.abs(left))
right = right / np.max(np.abs(right))

# make sure each of the left and right channel have equal length
len_left = len(left)
len_right = len(right)
if len_left > len_right:
    right = np.pad(right, (0, len_left - len_right), 'constant', constant_values = (0.0, 0.0))
else:
    left = np.pad(left, (0, len_left - len_right), 'constant', constant_values = (0.0, 0.0))

# convert back to stereo file
crosstalk_signal = np.zeros((2, len(left)))
crosstalk_signal[0] = left
crosstalk_signal[1] = right

In [15]:
ipd.Audio(crosstalk_signal, rate=sr)